In [1]:
import hdf5_getters as GETTERS
import h5py
import glob
from pyspark.sql import SparkSession
import pyspark
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

import numpy as np
import hdf5_getters as GETTERS
import h5py
import glob
import pandas as pd

In [ ]:
# Localiza un archivo de canción
filename = glob.glob(
    r'dataset\A\A\A\TRAAAAW128F429D538.h5'
)[0]

# Abre el archivo
h5 = GETTERS.open_h5_file_read(filename)

# Obtén algunos datos
artist_name = GETTERS.get_artist_name(h5)
song_title = GETTERS.get_title(h5)
year = GETTERS.get_year(h5)
duration = GETTERS.get_duration(h5)

print(
    f"Artist: {artist_name}, Song: {song_title}, Year: {year}, Duration: {duration:.2f}s")

h5.close()

#### 1. Crear la sesión con SPARK

In [7]:
#Initialize spark:
# import findspark
# findspark.init()
# print(pyspark.__version__)
# findspark.init("C:\spark\spark-3.5.1-bin-hadoop3\spark-3.5.1-bin-hadoop3")


# spark = SparkSession.builder \
#     .appName("MSD Recommender") \
#     .master("spark://192.168.1.130:7077") \
#     # .config("spark.driver.host", "192.168.1.130") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.executor.memory", "12g") \
#     .config("spark.executor.cores", "3") \
#     .getOrCreate()


from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("MillionSongProcessing") \
#     .master("spark://spark-master:7077") \
#     .getOrCreate()
spark = SparkSession \
    .builder \
    .appName('XML ETL') \
    .master("local[*]") \
    .config('job.local.dir', 'file:/models/music-recommender-model') \
    .getOrCreate()

df = spark.range(10).toDF("number")
df.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
+------+



In [19]:
# Stop Spark session
spark.stop()

In [15]:
df_test = spark.range(10)  # 1 millón de filas
print(df_test.count())  # Debe devolver 1,000,000

10


#### Analizar el dataset

In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.types import StructType, StructField, DoubleType, StringType

import numpy as np
import hdf5_getters as GETTERS
import h5py
import glob
import pandas as pd

sc = spark.sparkContext

In [9]:
# Cargar ruta del dataset

# #En local
# dataset_path = "/opt/bitnami/spark/datasets/MillionSongSubset"
#En Empresa
dataset_path = "/opt/bitnami/spark/datasets/"
file_paths = glob.glob(f"{dataset_path}/**/*.h5", recursive=True)
#Usando solo 3 paths
# file_paths = file_paths[:100]
# print(file_paths)

In [11]:
# Lista para guardar los features
data = []
metadata_list = []  # OJO cambiamos el nombre de la lista de metadatos


def extract_features(path):
    try:
        h5 = GETTERS.open_h5_file_read(path)

        features = {
            'song_id': GETTERS.get_song_id(h5).decode('utf-8'),
            'title': GETTERS.get_title(h5).decode('utf-8'),
            'artist': GETTERS.get_artist_name(h5).decode('utf-8'),
            'tempo': float(GETTERS.get_tempo(h5)),
            'key': float(GETTERS.get_key(h5)),
            'mode': float(GETTERS.get_mode(h5)),
            'time_signature': int(GETTERS.get_time_signature(h5)),
            'loudness': float(GETTERS.get_loudness(h5)),
            'danceability': float(GETTERS.get_danceability(h5))
        }

        h5.close()

        return features

    except Exception as e:
        print(f"Error procesando el fichero: {path} --> {e}")
        return None, None


# Recorremos los ficheros uno a uno
for path in file_paths:
    result = extract_features(path)
    data.append(result)

# Convertimos las listas a pandas
df_pd = pd.DataFrame(data)

# Y lo convertimos a DataFrame de PySpark
df = spark.createDataFrame(df_pd)

print("Features extracted")

Features extracted


In [12]:
df.show(10)

+------------------+--------------------+--------------------+-------+----+----+--------------+--------+------------+
|           song_id|               title|              artist|  tempo| key|mode|time_signature|loudness|danceability|
+------------------+--------------------+--------------------+-------+----+----+--------------+--------+------------+
|SOZECOE12AB017E615|Diamonds Are A Gi...|     Gloria De Haven|100.296| 0.0| 0.0|             4| -14.088|         0.0|
|SOXNSGB12A8C1396F1|     Gotta Let It Go|          Elva Hsiao|121.968|11.0| 1.0|             4|   -6.01|         0.0|
|SOGXWRE12AC468BE24|      Paint It Black|       Chris Farlowe|146.913| 7.0| 0.0|             1| -11.027|         0.0|
|SOIJPPR12A6D4F3945|Requiem Mass_ K. ...|Choir & Great Sym...|109.051| 9.0| 1.0|             4| -15.586|         0.0|
|SOMUYIL12AB0184EE2|Don't Owe Me Nothin'|Sticky Fingaz & O...| 87.897| 1.0| 1.0|             4|  -9.532|         0.0|
|SOMSKPE12A6D4FA09D|The Matter With Y...|               

In [15]:
df_pd.to_parquet("/opt/bitnami/spark/parquet/", index=False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

#### Entrenar el Modelo

In [8]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

feature_cols = [
    'tempo',
    'loudness',
    'danceability',
    'key',
    'mode',
    'time_signature'
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vec")
df_assembled = assembler.transform(df)

scaler = StandardScaler(inputCol="features_vec", outputCol="scaled_features")
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

kmeans = KMeans(
    featuresCol='scaled_features',
    predictionCol='cluster_id',
    k=20,
    seed=42
)

#Cálculo de centroides
model = kmeans.fit(df_scaled)

#Ver en que centroide cae cada canción
df_with_cluster = model.transform(df_scaled)

IllegalArgumentException: tempo does not exist. Available: number

In [22]:
df_scaled.show(1)


+------------------+--------------------+---------------+-------+---+----+--------------+--------+------------+--------------------+--------------------+
|           song_id|               title|         artist|  tempo|key|mode|time_signature|loudness|danceability|        features_vec|     scaled_features|
+------------------+--------------------+---------------+-------+---+----+--------------+--------+------------+--------------------+--------------------+
|SOZECOE12AB017E615|Diamonds Are A Gi...|Gloria De Haven|100.296|0.0| 0.0|             4| -14.088|         0.0|[100.296,-14.088,...|[2.85058055569665...|
+------------------+--------------------+---------------+-------+---+----+--------------+--------+------------+--------------------+--------------------+
only showing top 1 row



#### Recomendación 

In [24]:
import numpy as np
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeansModel

# Supongamos que tienes ya definido:
# model, scaler_model, assembler, df_scaled

# Simulamos la nueva canción
new_song = {
    'tempo': 120.0,
    'loudness': -5.0,
    'danceability': 1000,
    'key': 2.0,
    'mode': 1.0,
    'time_signature': 4.0,
    'song_id': 'NEW_SONG_ID'
}


# Convertimos a Spark DataFrame
new_song_df_pd = pd.DataFrame([new_song])
new_song_df = spark.createDataFrame(new_song_df_pd)

#  Ensamblamos y escalamos usando el pipeline original
new_song_vec = assembler.transform(new_song_df)
new_song_scaled = scaler_model.transform(new_song_vec)

# Hacemos la predicción de cluster
new_song_pred = model.transform(new_song_scaled)
predicted_cluster = new_song_pred.select(
    "cluster_id").collect()[0]["cluster_id"]

print(f"La nueva canción ha sido asignada al cluster: {predicted_cluster}")

# Buscamos canciones del dataset original en el mismo cluster

# Si no habías guardado los clusters en el df original:
df_with_cluster = model.transform(df_scaled)

# Filtramos por el mismo cluster
similar_songs = df_with_cluster.filter(
    df_with_cluster.cluster_id == predicted_cluster)

# Mostramos algunas recomendaciones (por ejemplo 3)
similar_songs.select("song_id").limit(3).show()

La nueva canción ha sido asignada al cluster: 16
+------------------+
|           song_id|
+------------------+
|SOUGKGL12A58A7BA58|
|SOMHUTQ12AB018DEF1|
|SOFCZKJ12A6D4F5BF0|
+------------------+



In [14]:
metadata_df = pd.read_csv("songs_metadata.csv")

# Supongamos que estos son los song_ids recomendados que has obtenido de tu modelo
recommended_song_ids = [row['song_id']
                        for row in similar_songs.select("song_id").limit(3).collect()]


# Filtramos los metadatos para obtener nombre de la canción y artista
recommended_songs = metadata_df[metadata_df['song_id'].isin(
    recommended_song_ids)]

# Mostramos el resultado
print(recommended_songs[['song_id', 'title', 'artist']])

                song_id                              title           artist
0    SOZECOE12AB017E615  Diamonds Are A Girl's Best Friend  Gloria De Haven
84   SOPJEXC12A6D4FB529                Hot Dental Supplies      Marga Gomez
149  SOJXYFD12A8C143701         The Ferocious O' Flahertys       Joe Heaney


#### Guardar el modelo

In [15]:
from pyspark.ml import Pipeline

# Creamos el pipeline con los stages que usaste durante el entrenamiento
pipeline = Pipeline(stages=[assembler, scaler, kmeans])

# Lo ajustamos sobre el dataframe ya escalado
pipeline_model = pipeline.fit(df)

In [16]:
pipeline_model.write().overwrite().save("file:///models/music-recommender-model")

##### Guardar las canciones y a que cluster pertenecen

In [17]:
df_with_cluster.show(5)

+-------+--------+---------+----+----+--------------+------------------+--------------------+--------------------+------------------+-----------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+--------------------+----------+
|  tempo|loudness| duration| key|mode|time_signature|           song_id|               title|              artist|     timbre_mean_0|     timbre_std_0|      timbre_mean_1|      timbre_std_1|      timbre_mean_2|      timbre_std_2|      timbre_mean_3|      timbre_std_3|     timbre_mean_4|      timbre_std_4|      timbre_mean_5|      timbre_std_5|      timbre_mean_6|     

In [18]:
df_export = df_with_cluster.select("title", "artist", "cluster_id")
df_export.write.csv("songs_with_cluster", header=True, mode="overwrite")

In [19]:
df_export.toPandas().to_csv("songs_with_cluster.csv", index=False)

#### Cargar el Modelo

In [5]:
from pyspark.ml import PipelineModel

loaded_model = PipelineModel.load("file:///models/music-recommender-model")

In [6]:
# Crear nueva canción con timbre simulado
new_song = {
    'tempo': 120.0,
    'loudness': -5.0,
    'danceability': 60,
    'key': 2.0,
    'mode': 1.0,
    'time_signature': 4.0,
    'track_name': 'New Song',
    'artist_name': 'Unknown Artist'
}



# Convertir a DataFrame de Spark
df_new = spark.createDataFrame(pd.DataFrame([new_song]))

# Predecir cluster directamente con el pipeline
predicted = loaded_model.transform(df_new)
predicted_cluster = predicted.select("cluster_id").collect()[0]["cluster_id"]

print(f"La canción fue asignada al cluster {predicted_cluster}")

df_with_clusters = loaded_model.transform(df_scaled)

recommendations = df_with_clusters.filter(
    df_with_clusters.cluster_id == predicted_cluster
).select("track_name", "artist_name").limit(3)

recommendations.show()


IllegalArgumentException: duration does not exist. Available: tempo, loudness, danceability, key, mode, time_signature, track_name, artist_name

In [15]:
metadata_df = pd.read_csv("songs_metadata.csv")

# Supongamos que estos son los song_ids recomendados que has obtenido de tu modelo
recommended_song_ids = [row['song_id']
                        for row in similar_songs.select("song_id").limit(3).collect()]


# Filtramos los metadatos para obtener nombre de la canción y artista
recommended_songs = metadata_df[metadata_df['song_id'].isin(
    recommended_song_ids)]

# Mostramos el resultado
print(recommended_songs[['song_id', 'title', 'artist']])

               song_id           title              artist
19  SOQBCSM12AC4687CDE  Pete's Crusade  Light Of The World
